# A full model

We will now build a full `simpy` process model.  In this example we will build a 111 call centre queuing model.  We will include random arrivals and resources.  We will keep this simple, and gradually add in detail and flexibility to our design.

## 1. Imports

In [1]:
import simpy
import numpy as np
import itertools

## 2. Problem background

Call operators in an 111 (urgent care) service receive calls at a rate of 100 per hour. Call length can be represented by a triangular distribution.  Calls last between 5 minutes and 15 minutes. Most calls last 7 minutes. There are 13 call operators.  

> We will build a `simpy` model of the process.

## 3. `simpy` resources

To model the call centre we need to introduce a **resource**.  If a resource is not available then a process will pause.  We create a resource as follows:

```python
operators = simpy.Resource(env, capacity=20)
```

When we want to request a resource in our process we create a with block as follows:

```python
with operators.request() as req:
    yield req
```

This tells simpy that your process needs an operator resource to progress.  The code will pause until a resource is yielded. This gives us our queuing effect.  If a resource is not available immediately then the process will wait until one becomes available.  We will practice this a lot in the course so do not worry if this does not make sense immediately.

## 4. The service function.

We will first build the service function. We need to do this because our arrival/generator function will be required to call it.

We need to include the following logic:

1. Request and if necessary wait for a call operator
2. Undergo phone triage (a delay). This is a sample from the Triangular distribution.
3. Exit the system.

We will build this as a simple python function that we will call `service`.  Each patient caller that arrives to the simulation will execute service as a `simpy` process.  We will pass a unique patient identifier, a pool of operator resources and the environment to the function.

In [2]:
def service(identifier, operators, env, service_rng):
    '''
    Simulates the service process for a call operator

    1. request and wait for a call operator
    2. phone triage (triangular)
    3. exit system
    
    Params:
    ------
    
    identifier: int 
        A unique identifer for this caller
        
    operators: simpy.Resource
        The pool of call operators that answer calls
        These are shared across resources.
        
    env: simpy.Environment
        The current environent the simulation is running in
        We use this to pause and restart the process after a delay.

    service_rng: numpy.random.Generator
        The random number generator used to sample service times
    
    '''
    # record the time that call entered the queue
    start_wait = env.now

    # request an operator
    with operators.request() as req:
        yield req

        # record the waiting time for call to be answered
        waiting_time = env.now - start_wait
        print(f'operator answered call {identifier} at ' \
              + f'{env.now:.3f}')

        # sample call duration.
        call_duration = service_rng.triangular(left=5.0, mode=7.0,
                                               right=10.0)
        
        # schedule process to begin again after call_duration
        yield env.timeout(call_duration)

        # print out information for patient.
        print(f'call {identifier} ended {env.now:.3f}; ' \
              + f'waiting time was {waiting_time:.3f}')

## 5. The generator function.

The generator function is very similar to the pharamacy example.  

> **Notice the pattern**. For most models you can just cut, paste and modify code you have used before.

In [3]:
def arrivals_generator(env, operators):
    '''
    IAT is exponentially distributed

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    operators: simpy.Resource
        the pool of call operators.
    '''
    # create the arrival process rng 
    arrival_rng = np.random.default_rng()
    
    # create the service rng that we pass to each service process created
    service_rng = np.random.default_rng()
    
    # use itertools as it provides an infinite loop 
    # with a counter variable that we can use for unique Ids
    for caller_count in itertools.count(start=1):

        # 100 calls per hour (sim time units = minutes). 
        inter_arrival_time = arrival_rng.exponential(60/100)
        yield env.timeout(inter_arrival_time)

        print(f'call arrives at: {env.now:.3f}')

        # create a new simpy process for serving this caller.
        # we pass in the caller id, the operator resources, env, and the rng
        env.process(service(caller_count, operators, env, service_rng))

## 6. Run the model

In [4]:
# model parameters
RUN_LENGTH = 100
N_OPERATORS = 13

# create simpy environment and operator resources
env = simpy.Environment()
operators = simpy.Resource(env, capacity=N_OPERATORS)

env.process(arrivals_generator(env, operators))
env.run(until=RUN_LENGTH)
print(f'end of run. simulation clock time = {env.now}')

call arrives at: 0.206
operator answered call 1 at 0.206
call arrives at: 1.467
operator answered call 2 at 1.467
call arrives at: 1.739
operator answered call 3 at 1.739
call arrives at: 2.400
operator answered call 4 at 2.400
call arrives at: 2.580
operator answered call 5 at 2.580
call arrives at: 2.755
operator answered call 6 at 2.755
call arrives at: 3.846
operator answered call 7 at 3.846
call arrives at: 5.182
operator answered call 8 at 5.182
call arrives at: 5.963
operator answered call 9 at 5.963
call arrives at: 6.631
operator answered call 10 at 6.631
call arrives at: 7.189
operator answered call 11 at 7.189
call 1 ended 7.567; waiting time was 0.000
call arrives at: 8.082
operator answered call 12 at 8.082
call 3 ended 8.435; waiting time was 0.000
call arrives at: 8.486
operator answered call 13 at 8.486
call arrives at: 8.802
operator answered call 14 at 8.802
call 2 ended 9.167; waiting time was 0.000
call 5 ended 9.546; waiting time was 0.000
call 6 ended 9.776; waiti